In [69]:
import requests 
import urllib
import pandas as pd

PAGE_SIZE = 500
#DHVXC_REST_DECODED_URL = 'https://de.dhv-xc.de/api/fli/flights?fkcat[]=1&fkto[]={place_id}&navpars={"start":{start},"limit":{limit},"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}'

places = {
    'Metzingen':11185,
    'Rammelsberg': 9427,
    'Estorf': 11001,
    'Leese': 10746,
    'Porta': 9712,
    'Lüdingen':9759,
    'Brunsberg': 9844,
    'Kella': 9521,
    'Börry': 9403
}


results = []
#for place in ['Metzingen','Rammelsberg']:
for place in places:
    limit = PAGE_SIZE
    for start in [0, PAGE_SIZE]:
        query = {"navpars":{"start":start,"limit":limit,"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}}
        # for some reason, " is replaced by ' and spaces are added which is replaced by +
        decoded_url = f"https://de.dhv-xc.de/api/fli/flights?s?fkcat%5B%5D=1&fkto%5B%5D={places[place]}&{urllib.parse.urlencode(query,quote_via=urllib.parse.quote_plus).replace('%27', '%22').replace('+', '')}"
        #print(decoded_url)
        r = requests.get(decoded_url)
        if r.status_code==200:
            response = r.json()
            df = pd.DataFrame(response['data'])
            #print(df.columns.values)
            results.extend(df[['FlightDate', 'TakeoffWaypointName' , 'Glider' ,'FlightDuration']].values)


df_results = pd.DataFrame(results,columns=['FlightDate', 'Takeoff' , 'Glider' ,'Duration'])
print(len(df_results))
df_results.tail()


9000


,FlightDate,Takeoff,Glider,Duration
8995,2021-09-09,Börry - Am Hellberg,SummitXC4 - M [LTF B],1184
8996,2021-09-09,Börry - Am Hellberg,Bibeta6 - XL [LTF B],1165
8997,2021-09-09,Börry - Am Hellberg,Ion 6 L,1243
8998,2021-09-09,Börry - Am Hellberg,Xalps4 - S [LTF D],2657
8999,2021-09-09,Börry - Am Hellberg,SummitXC4 - M [LTF B],1285


In [87]:
from datetime import datetime

#df_results[['FlightDate', 'Takeoff']].groupby('Takeoff').aggregate(['min','max'])
df_results = df_results[df_results['FlightDate']>'2023-01-01']
df_results['Takeoff'] = df_results.apply(lambda row: row['Takeoff'].split('-')[0].strip() ,axis=1)
df_gr= df_results[['FlightDate', 'Takeoff','Duration']].groupby(by=['Takeoff','FlightDate' ]).aggregate(count=('Duration','count')).reset_index()
#df_gr.tail()
df_pvt = pd.pivot_table(df_gr,index='FlightDate',columns='Takeoff',aggfunc='sum', fill_value=0)
df_pvt['Weekday'] = df_pvt.apply(lambda row: datetime.strptime(row.name,'%Y-%m-%d').strftime("%A"), axis=1)
filter = df_pvt['Weekday'].isin(['Sunday'])
df_pvt[(df_pvt[('count','Metzingen')]==0) & (filter)]

count                                                         \
Takeoff    Brunsberg Börry Estorf Kella Berg Leese Lüdingen Metzingen Porta   
FlightDate                                                                    
2023-01-08         0     8      0          0     0        0         0     3   
2023-02-19         0     0      0          0     0        0         0     0   
2023-02-26         0     0      0          0     0        0         0     0   
2023-04-09         5     0      0          4    10        0         0     0   
2023-05-07        12     0      0          0     0        0         0     0   
2023-05-14         1     0      0          6     0        0         0     2   
2023-06-11        24     0      0          1     0        0         0     0   
2023-07-09         0     0      0          0     0        3         0     1   
2023-08-13         0     0      0          0     0        0         0     8   
2023-08-20         0     0      0          0     0        0         0     0   
2023-09-03         0     0     13          0     0        8         0     0   
2023-09-10         0     0     12          0     0        0         0     0   
2023-09-17         0    12      0          0     0        2         0    10   
2023-10-22         0     0      0          0     0        0         0     1   
2023-11-12         0     0      0          0     0        0         0    25   
2023-12-03         0     0      0          0     0        0         0     7   
2024-01-28         0    10      2          0     0        0         0    14   
2024-02-18         0     2      0          0     0        0         0     0   
2024-02-25         0     4      0          0     0        0         0     9   
2024-03-03        17     0      0          8     0        0         0     0   
2024-03-17         0     0      0          0    10        0         0     1   
2024-04-28         0     1      0          0     0        0         0     0   
2024-05-12        40     0      0          6     0        0         0     0   
2024-06-09         0     0      0          0     0        0         0     1   
2024-06-16         0     1      0          0     0        0         0     0   
2024-06-23         0     0      0          0    11        0         0     0   
2024-07-28         0     0      0          0     0        0         0     0   
2024-08-11         0     0     10          0     0        0         0     0   
2024-09-01        18     0      0          3     0        3         0     0   
2024-09-08         0     0      0          0     0        0         0     0   
2024-09-15         0     0      0          0     0        0         0     0   

                       Weekday  
Takeoff    Rammelsberg          
FlightDate                      
2023-01-08           0  Sunday  
2023-02-19           2  Sunday  
2023-02-26           1  Sunday  
2023-04-09           0  Sunday  
2023-05-07           0  Sunday  
2023-05-14           0  Sunday  
2023-06-11           0  Sunday  
2023-07-09           0  Sunday  
2023-08-13           1  Sunday  
2023-08-20          16  Sunday  
2023-09-03          27  Sunday  
2023-09-10          19  Sunday  
2023-09-17           0  Sunday  
2023-10-22           0  Sunday  
2023-11-12           0  Sunday  
2023-12-03           0  Sunday  
2024-01-28           0  Sunday  
2024-02-18           0  Sunday  
2024-02-25           0  Sunday  
2024-03-03           0  Sunday  
2024-03-17           2  Sunday  
2024-04-28           0  Sunday  
2024-05-12           0  Sunday  
2024-06-09           0  Sunday  
2024-06-16           0  Sunday  
2024-06-23          28  Sunday  
2024-07-28          10  Sunday  
2024-08-11          56  Sunday  
2024-09-01           0  Sunday  
2024-09-08          11  Sunday  
2024-09-15           6  Sunday